## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [ ]:
from nltk import download

download('movie_reviews')

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [ ]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    if(len(review)<= 5000): # since Amazon AWS Comprehend has record limit of 5000 filtering lonner reviews
        reviews_pos.append(review)
print("Positive Reviews Count: " + str(len(reviews_pos)))

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    if(len(review)<= 5000): # since Amazon AWS Comprehend has record limit of 5000 filtering lonner reviews
        reviews_neg.append(review)
print("Negative Reviews: " + str(len(reviews_neg)))

In [ ]:
#getting Sentiment weights
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        for k, v in obj.items():
            if isinstance(v, (dict, list)):
                extract(v, arr, key)
            elif k == key:
                arr.append(v)
        return arr

    results = extract(obj, arr, key)
    return results

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [ ]:
import boto3
import json

# Classify all the review sentiments as Positive or Negative
def score_review(review):
    comprehend = boto3.client(service_name='comprehend', region_name='ap-southeast-1')
    sentimentrec = comprehend.detect_sentiment(Text=review, LanguageCode='en')
    ambiguous_sentiments =["NEUTRAL", "MIXED"]

     # Sentiment is  ambiguous - Mixed or Neutral
    if(sentimentrec['Sentiment'] in ambiguous_sentiments):
        pos_weight = extract_values(sentimentrec, 'Positive')[0]
        neg_weight = extract_values(sentimentrec, 'Negative')[0]
        if(pos_weight >= neg_weight):
            user_sentiment = 'pos'
        else:
            user_sentiment = 'neg'
            
    # Sentiment is clear - Positive or Negative
    if(sentimentrec['Sentiment'] == 'POSITIVE'):
        user_sentiment = 'pos'
    else:
        user_sentiment = 'neg'
    return user_sentiment

### Score each review

Now, we can use the function you defined to score each of the reviews

In [ ]:
results_pos = []
for review in reviews_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
for review in reviews_neg:
    result = score_review(review)
    results_neg.append(result)

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [ ]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))